In [1]:
#STEP 1 - import packages
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

import plotly as plotly
import plotly.plotly as py
import plotly.graph_objs as go
import plotly.figure_factory as ff

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
plotly.tools.set_credentials_file(username='martijnvanvlie', api_key='nDD1QdIOq3rOEO51bazE')

#set specific color
pblue = 'rgb(31,119,180)'
porange = 'rgb(255,127,14)'

In [48]:
#STEP 2 - Load in data
df_traffic = pd.read_csv('Traffic_segment_data.csv', encoding='utf-8',sep = ",")
df_BMSS_traffic_link = pd.read_csv('Bridge_data_with_link_to_traffic.csv',encoding='utf-8',sep = ",")

#calculate segment score (we weigth vulnerability and criticality equally)
df_traffic["segment_score"] = ""
df_traffic["Road_cat"] = ""
df_traffic["segment_score"] = df_traffic["segment_vulnerability_normalized"] * df_traffic["Economic_Traffic_normalized"]

#get type of road to plot in scatter
df_traffic["Road_cat"] =  df_traffic['Road_Name'].astype(str).str[0]

#get top-10 segments of all segments of all roads
df_traffic = df_traffic.sort_values(by='segment_score',ascending = False)
top_points_df = df_traffic[0:10]
top_points_df["numeric"] = ""

for row in range(len(top_points_df)):
    if top_points_df.Road_cat.iloc[row,] == "N":
        top_points_df.numeric.iloc[row,] = 1
    if top_points_df.Road_cat.iloc[row,] == "R":
        top_points_df.numeric.iloc[row,] = 2

#interactive transport scatter plot data
df = pd.read_csv('Traffic_segment_data.csv')
df.drop(df.columns[0:1], axis=1,inplace = True)
df.drop('segment_vulnerability_length', axis = 1, inplace = True)
df.drop('missing_segment', axis = 1, inplace = True)
df=df.rename(columns = {'segment_vulnerability_normalized':'Vulnerability'})
df=df.rename(columns = {'Economic_Traffic_normalized':'Criticality'})
df.Criticality = df.Criticality.apply(pd.to_numeric)
df.dropna(inplace=True)


df_BMSS_traffic_link

,Unnamed: 0,road,km,type,LRPName,name,length,condition,structureNr,roadName,...,zone,circle,division,sub-division,lat,lon,EstimatedLoc,average_delay,Road_segment_no,Road_segment_label
0,0,N1,1.800,Box Culvert,LRP001a,.,11.30,A,117861,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,...,Dhaka,Dhaka,Narayanganj,Narayanganj-1,23.698739,90.458861,interpolate,NaN,N1-2R,Int.with Z1101 - Signboard (Left) R111 (Right)
1,1,N1,4.925,Box Culvert,LRP004b,.,6.60,A,117862,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,...,Dhaka,Dhaka,Narayanganj,Narayanganj-1,23.694664,90.487775,interpolate,NaN,N1-3,Signboard - Shimrail (Left)R110 (Left)
2,2,N1,8.976,PC Girder Bridge,LRP008b,Kanch pur Bridge.,394.23,A,119889,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,...,Dhaka,Dhaka,Narayanganj,Narayanganj-1,23.705060,90.523214,interpolate,NaN,N1-5,Katchpur - Madanpur (Left)N105 (Left)
3,3,N1,10.880,Box Culvert,LRP010b,NOYAPARA CULVERT,6.30,A,112531,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,...,Dhaka,Dhaka,Narayanganj,Vitikandi,23.694391,90.537574,interpolate,NaN,N1-5,Katchpur - Madanpur (Left)N105 (Left)
4,4,N1,10.897,Box Culvert,LRP010c,ADUPUR CULVERT,6.30,A,112532,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,...,Dhaka,Dhaka,Narayanganj,Vitikandi,23.694302,90.537707,interpolate,NaN,N1-5,Katchpur - Madanpur (Left)N105 (Left)
5,5,N1,11.296,Box Culvert,LRP011a,NAYABARI KASPUR BOX CULVERT,8.30,A,101110,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,...,Dhaka,Dhaka,Narayanganj,Vitikandi,23.692360,90.540918,interpolate,NaN,N1-5,Katchpur - Madanpur (Left)N105 (Left)
6,6,N1,12.239,Box Culvert,LRP012a,KHAS PARA BOX CULVERT,9.30,A,101117,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,...,Dhaka,Dhaka,Narayanganj,Vitikandi,23.688412,90.548559,interpolate,NaN,N1-6,Madanpur - Langalband (Left)Z1061 (Left)
7,7,N1,12.253,Box Culvert,LRP012b,DAWAN BAG BOX CULVERT,6.10,A,101119,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,...,Dhaka,Dhaka,Narayanganj,Vitikandi,23.688320,90.548650,interpolate,NaN,N1-6,Madanpur - Langalband (Left)Z1061 (Left)
8,8,N1,12.660,PC Girder Bridge,LRP013a,Madanpur Bridge.(L),27.50,A,119897,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,...,Dhaka,Dhaka,Narayanganj,Vitikandi,23.685583,90.551208,interpolate,NaN,N1-6,Madanpur - Langalband (Left)Z1061 (Left)
9,9,N1,12.660,PC Girder Bridge,LRP013a,MADAN PUR (R),26.30,A,109841,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,...,Dhaka,Dhaka,Narayanganj,Vitikandi,23.685583,90.551208,interpolate,NaN,N1-6,Madanpur - Langalband (Left)Z1061 (Left)


In [59]:
df_traffic_assignment_4 = df_traffic.sort_values(by='segment_score', ascending=False)
df_traffic_assignment_4 = df_traffic_assignment_4[df_traffic_assignment_4["Road_Name"]=="N1"]
df_traffic_assignment_4 = df_traffic_assignment_4[0:5]
df_traffic_assignment_4 = df_traffic_assignment_4.sort_values(by='segment_score', ascending=False)
df_traffic_assignment_4 = df_traffic_assignment_4[['Road_Name','Link_No','Link_Name','Start LRP','End LRP','Total_A','Total_B','Total_C','Total_D' ,'segment_score']]


Top_five_segments = df_traffic_assignment_4["Link_No"]
Top_five_segments = Top_five_segments.values.tolist()

df_most_important_bridges = df_BMSS_traffic_link[df_BMSS_traffic_link["Road_segment_no"].isin(Top_five_segments)]
df_most_important_bridges = df_most_important_bridges[['name','LRPName','Road_segment_no','condition' ]]

df_most_important_bridges.to_csv('data_five_most_important_segments', sep=',')

['N1-36', 'N1-7', 'N1-69', 'N1-12', 'N1-24']

In [3]:
#Step 3 - Interactive transport mode plot
hover_text = []
# Iterate through all the rows and create the text we want to show when hovering over a certain data point. Add this to the column Text
for index, row in df.iterrows():
    hover_text.append(('Criticality: {criticality}<br>'+
                      'Vulnerability: {vulnerability}<br>' + 
                       'Road name: {roadname}<br>' + 
                       'Start LRP: {startLRP}<br>' + 
                       'End LRP: {endLRP}'
                      ).format(criticality=round(row['Criticality'],2),
                               vulnerability=round(row['Vulnerability'],2),
                               roadname=row['Road_Name'],
                               startLRP=row['Start LRP'],
                               endLRP=row['End LRP'])
                     )
df['Text'] = hover_text
data = []
# Get a list of all different modes of transport
modes = list(df)[12:28]
# Create the same number of traces, so we can switch on and off the different modes of transport
traces = ['trace' + str(i) for i in range(1,len(modes)+1)] 
# Define the traces with on the x-axis the Criticality and on the y-axis the Vulnerability 
# and make the size correspond to the density of that mode of transport
for trace, mode in zip(traces, modes):
    sizeref = 2.*max(df[mode])/(100**2)
    trace = go.Scatter(
        x=df['Criticality'], 
        y=df['Vulnerability'],
        mode='markers',
        name=mode,
        text=df['Text'],
        marker=dict(
            symbol='circle',
            sizemode='area',
            sizeref=sizeref,
            size=df[mode]/4,
            line=dict(
                width=2
            ),
        )
    )
    data.append(trace)
    
data = data
# Add the layout to the map. This includes the legend, title and the settings for the axes
layout = go.Layout(
    hovermode = 'closest',
    title='Criticality vs. vulnerability for all road segments per mode of transport',
    annotations=[
        dict(
            x=1.12,
            y=1.05,
            align="right",
            valign="top",
            text='Modes of transport',
            showarrow=False,
            xref="paper",
            yref="paper",
            xanchor="center",
            yanchor="top",
        )],
    xaxis=dict(
        title='Criticality',
        gridcolor='rgb(255, 255, 255)',
        range=[0,1],
        zerolinewidth=1,
        ticklen=5,
        gridwidth=2,
    ),
    yaxis=dict(
        title='Vulnerability',
        gridcolor='rgb(255, 255, 255)',
        range=[0,1],
        zerolinewidth=1,
        ticklen=5,
        gridwidth=2,
    ),
    paper_bgcolor='rgb(243, 243, 243)',
    plot_bgcolor='rgb(243, 243, 243)',
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig,visible = 'legendonly')

In [4]:
#STEP 4 - CREATE SCATTERPLOT CRITICALITY/VULNERABIILTY/ROAD CATEGORIE BASED ON SCORES (TOP 10)
#make two traces, one for R roads one for N roads
trace_1_x = top_points_df[top_points_df["Road_cat"] == "R"]
trace_1_x = trace_1_x["segment_vulnerability_normalized"]
trace_1_x = trace_1_x.values.tolist()

trace_1_y = top_points_df[top_points_df["Road_cat"] == "R"]
trace_1_y = trace_1_y["Economic_Traffic_normalized"]
trace_1_y = trace_1_y.values.tolist()

trace_2_x = top_points_df[top_points_df["Road_cat"] == "N"]
trace_2_x = trace_2_x["segment_vulnerability_normalized"]
trace_2_x = trace_2_x.values.tolist()

trace_2_y = top_points_df[top_points_df["Road_cat"] == "N"]
trace_2_y = trace_2_y["Economic_Traffic_normalized"]
trace_2_y = trace_2_y.values.tolist()

#setup structure of traces
trace1 = {"x": trace_1_y,
          "y": trace_1_x, 
          "marker": {"color": porange, "size": 12}, 
          "mode": "markers", 
          "name": "R roads", 
          "type": "scatter"}

trace2 = {"x": trace_2_y, 
          "y": trace_2_x, 
          "marker": {"color": pblue, "size": 12}, 
          "mode": "markers", 
          "name": "N roads", 
          "type": "scatter", 
}

data = [trace1, trace2]
#setup layout of the plot
layout = {"title": "Top 10 road segments based on Criticalility and Vulnerability", 
          "xaxis": {"title": "Criticality (normalised)", }, 
          "yaxis": {"title": "Vulnerability (normalised)"}}

fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

In [5]:
#Plot scatter plot of segment score
#Get list names and scores of each segment
list_of_road_segments = top_points_df.Link_No.tolist()
list_of_scores = top_points_df.segment_score.tolist()

#setup structure of plot
data = [go.Scatter(
            x=list_of_road_segments,
            y=list_of_scores,
            name='Absolute',
            mode = "markers",
            marker=dict(line=dict(color=porange, width=10)))]
#setup layout of plot (x axis and y axis)
layout = go.Layout(
            title='Segment score of top 10 road segments',
            yaxis=dict(title= 'Segment score', range = [0,0.2]),
            xaxis=dict(title= 'Road label'))
fig = dict(data=data, layout=layout)
py.iplot(fig)